<a href="https://colab.research.google.com/github/Nicordaro/Project_MLDL/blob/master/Project_MLDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Politecnico di Torino**

**01TXFSM - Machine learning and Deep learning**

**Incremental Learning in Image Classification**

**Cordaro Nicolò - sMATRICOLA**

**Di Nepi Marco - sMATRICOLA**

**Falletta Alberto - s277971**


In [1]:
!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'

**Imports**

In [0]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import resnet18

from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
from google.colab import drive

**Arguments**

In [0]:
DEVICE = 'cuda' # 'cuda' or 'cpu'

NUM_CLASSES = 7

BATCH_SIZE = 256     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3            # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 30      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 20       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

**Pre-processing**

In [0]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # https://github.com/pytorch/examples/blob/master/imagenet/main.py
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])                                 
])

**Prepare Dataset**

CIFAR100 has 100 classes containing 600 images each. There are 500 training images and 100 testing images per class. The 100 classes in the CIFAR-100 are grouped into 20 superclasses. Each image comes with a "fine" label (the class to which it belongs) and a "coarse" label (the superclass to which it belongs).

The dataset is divided into five training batches and one test batch, each with 10000 images. The test batch contains exactly 1000 randomly-selected images from each class. The training batches contain the remaining images in random order, but some training batches may contain more images from one class than another.

Each of the downloaded files is a Python "pickled" object produced with cPickle.

In [18]:
from torchvision.datasets import CIFAR100

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

DATA_DIR = './CIFAR100'

if not os.path.isdir('./CIFAR100'):
  dataset = CIFAR100(DATA_DIR, train=True, transform=None, target_transform=None, download=True)

train = unpickle(os.path.join(DATA_DIR, "cifar-100-python", "test"))
print(train)

{b'filenames': [b'volcano_s_000012.png', b'woods_s_000412.png', b'seal_s_001803.png', b'mushroom_s_001755.png', b'adriatic_sea_s_000653.png', b'tulipa_clusiana_s_000175.png', b'camel_s_001052.png', b'mourning_cloak_s_000143.png', b'cirrostratus_s_000223.png', b'eating_apple_s_000763.png', b'adriatic_s_000380.png', b'skunk_s_001175.png', b'streetcar_s_000382.png', b'sounding_rocket_s_000010.png', b'lamp_s_001350.png', b'king_of_beasts_s_001221.png', b'tulipa_clusiana_s_000186.png', b'timber_wolf_s_000622.png', b'rose_s_002192.png', b'valencia_orange_s_001542.png', b'mountain_rose_s_000209.png', b'mount_s_002678.png', b'striped_skunk_s_001304.png', b'dinosaur_s_000326.png', b'chimpanzee_s_000134.png', b'oilcan_s_000656.png', b'keyboard_s_001529.png', b'safety_bike_s_000390.png', b'bike_s_000658.png', b'rose_s_000120.png', b'chaise_s_001826.png', b'fishplate_s_000669.png', b'motor_mower_s_000528.png', b'sea_turtle_s_000882.png', b'elaeis_guineensis_s_000412.png', b'mackerel_shark_s_001042